<h1>總整理

In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


In [2]:
np.__version__,pd.__version__

('1.17.4', '0.25.3')

In [3]:
# pd.set_option("display.max_rows",None)
# pd.set_option("display.max_columns",None)

##### 讀取資料

In [4]:
# 家
# prev_data = pd.read_csv('previous_application.csv')
# POS_data = pd.read_csv('POS_CASH_balance.csv')
# 學校
prev_data = pd.read_csv('..\\..\\Desktop\\home-credit-default-risk\\previous_application.csv')
POS_data = pd.read_csv('..\\..\\Desktop\\home-credit-default-risk\\POS_CASH_balance.csv')
# 組長
# prev_data = pd.read_csv('../../datasets/homecdt_eda/previous_application.csv')
# POS_data = pd.read_csv('../../datasets/homecdt_eda/POS_CASH_balance.csv')


### POS_CASH_balance.csv特徵工程
---
   

In [5]:
# 2020-01-16

# Total_Months --> 總還款期數
# MONTHS_BALANCE_start --> 貸款開始時間
# MONTHS_BALANCE_finish --> 貸款結束時間
# CNT_INSTALMENT_max --> 最大申請期數(擬定合約期數)
# CNT_INSTALMENT_min --> 最小申請期數(中間改合約)
# CNT_INSTALMENT_median --> 申請期數中位數
# Delay_Rate --> 遲繳比例(ex 共四期 有一期遲繳就是1/4)
# SK_DPD_max --> 貸款愈期最大天數
# SK_DPD_mean --> 貸款逾期平均數
# Contract_Change -- > 提早繳完 且 不在目前進行中的貸款
# Contract_Change_count --> 合約縮短了多少期
# CNT_INSTALMENT/Total_Months_rate --> 申請期數與總還款期數比例
# Contract_Change_rate(CNT_INSTALMENT) --> 縮短了多少期除以申請期數
# Contract_Change_rate(Total_Months) --> 縮短了多少期除以總還款期數

# 將有特殊狀態的欄位新增 (onehot)
categorical_columns = []
for name in POS_data:
    if POS_data[name].dtype=='object':
        categorical_columns.append(name)
POS_data = pd.get_dummies(POS_data, columns = categorical_columns)

POS_data.rename(columns={'NAME_CONTRACT_STATUS_Active':'Active',
                     'NAME_CONTRACT_STATUS_Amortized debt':'Amortized debt',
                    'NAME_CONTRACT_STATUS_Approved':'Approved',
                    'NAME_CONTRACT_STATUS_Canceled':'Canceled',
                    'NAME_CONTRACT_STATUS_Completed':'Completed',
                    'NAME_CONTRACT_STATUS_Demand':'Demand',
                    'NAME_CONTRACT_STATUS_Returned to the store':'Returned to the store',
                    'NAME_CONTRACT_STATUS_Signed':'Signed',
                    'NAME_CONTRACT_STATUS_XNA':'XNA'},inplace=True)

# 新增欄位

POS_data['Delay']=(POS_data['SK_DPD']>0).replace(True,1) # 該期貸款有愈期，1為有延遲，0為沒延遲
POS_data['SK_DPD_mean']=POS_data['SK_DPD'] # 要算平均
POS_data['CNT_INSTALMENT_min']=POS_data['CNT_INSTALMENT'] # 申請期最小值
POS_data['CNT_INSTALMENT_median']=POS_data['CNT_INSTALMENT'] # 申請期的中位數
POS_data['MONTHS_BALANCE_start']=POS_data['MONTHS_BALANCE'] # 貸款開始時間
POS_data['MONTHS_BALANCE_finish']=POS_data['MONTHS_BALANCE'] # 貸款結束時間


# 創造新的欄位 
num_aggregations = {
        'MONTHS_BALANCE':'count',
        'MONTHS_BALANCE_start':'min',
        'MONTHS_BALANCE_finish':'max', 
        'CNT_INSTALMENT' : 'max',
        'CNT_INSTALMENT_min':'min',
        'CNT_INSTALMENT_median':'median',
        'Delay':'mean',
        'SK_DPD':'max',
        'SK_DPD_mean':'mean',
        'Completed':'max',
        'Active':'sum',
        'Signed':'sum',
        'Demand':'sum',
        'Returned to the store':'sum',
        'Approved':'sum',
        'Amortized debt':'sum',
        'Canceled':'sum',
        'XNA':'sum'
    }

POS_data_1 = POS_data.groupby(['SK_ID_CURR',
                                    'SK_ID_PREV']).agg({**num_aggregations})
# 做更名
POS_data_1.rename(columns={'MONTHS_BALANCE':'Total_Months',
                      'Delay':"Delay_Rate",
                      'SK_DPD':'SK_DPD_max',
                      'CNT_INSTALMENT':'CNT_INSTALMENT_max'},inplace=True)
# 重新排列
POS_data_1.reset_index(level=('SK_ID_CURR',
                          'SK_ID_PREV'),inplace=True)

# 提早繳完 且 不在目前進行中的貸款
POS_data_1['Contract_Change'] = ((POS_data_1['Total_Months'] < POS_data_1 ['CNT_INSTALMENT_max']) & (POS_data_1['Completed'] != 0)).replace(True,1)
# 合約縮短了多少期
POS_data_1['Contract_Change_count'] = POS_data_1['CNT_INSTALMENT_max']-POS_data_1['CNT_INSTALMENT_min']
# 申請期數與總還款期數比例
POS_data_1['CNT_INSTALMENT/Total_Months_rate'] = POS_data_1['CNT_INSTALMENT_max']/POS_data_1['Total_Months']
# 縮短了多少期除以申請期數
POS_data_1['Contract_Change_rate(CNT_INSTALMENT)'] = POS_data_1['Contract_Change_count']/POS_data_1['CNT_INSTALMENT_max']
# 縮短了多少期除以總還款期數
POS_data_1['Contract_Change_rate(Total_Months)'] = POS_data_1['Contract_Change_count']/POS_data_1['Total_Months']
# 狀態發生期數除以總還款期數
POS_data_1['Active'] = POS_data_1['Active']/POS_data_1['Total_Months']
POS_data_1['Signed'] = POS_data_1['Signed']/POS_data_1['Total_Months']
POS_data_1['Demand'] = POS_data_1['Demand']/POS_data_1['Total_Months']
POS_data_1['Returned to the store'] = POS_data_1['Returned to the store']/POS_data_1['Total_Months']
POS_data_1['Approved'] = POS_data_1['Approved']/POS_data_1['Total_Months']
POS_data_1['Amortized debt'] = POS_data_1['Amortized debt']/POS_data_1['Total_Months']
POS_data_1['Canceled'] = POS_data_1['Canceled']/POS_data_1['Total_Months']
POS_data_1['XNA'] = POS_data_1['XNA']/POS_data_1['Total_Months']
# POS_data_1 = POS_data_1.drop(['SK_ID_CURR'],axis=1)

<h1> 將POS_CASH_balance.csv和installments_payments.csv併到previous_application.csv

In [6]:
# 同整installments_payments、POS_CASH_balance、credit_card_balance

prev_comb_data = pd.read_csv('..\\..\\Desktop\\home-credit-default-risk\\previous_application_w_installment.csv')

result = pd.merge(prev_comb_data, POS_data_1,how='outer')
# result.to_csv('..\\..\\Desktop\\home-credit-default-risk\\previous_application_w_installment_w_POS.csv')


# 家裡
# prev_comb_data = pd.read_csv('previous_application_w_installment.csv')

# result = pd.merge(prev_comb_data, POS_data_1,how='outer')
# result.to_csv('previous_application_w_installment_w_POS.csv')


c:\program files\python37\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (2,8,10,15,16,18,19,20,21,22,23,24,25,27,29,30) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<h1>異常值處理

In [7]:
# Prev時間異常值，另外新增欄位

result_test = result
issue_data = ['DAYS_FIRST_DRAWING','DAYS_FIRST_DUE','DAYS_LAST_DUE_1ST_VERSION','DAYS_LAST_DUE','DAYS_TERMINATION']
for name in issue_data:    
#     rename = name + "_ANOM"
#     data[rename] = data[name] == 365243
    result_test[name].replace({365243: np.nan}, inplace = True)
    
# SELLERPLACE_AREA 分行資料有接近一半是-1值，而其他皆為正整數，懷疑是未知
result_test['SELLERPLACE_AREA'].replace({-1: np.nan}, inplace = True)


# Prev將缺失值大於60的欄位去除
miss_value_percent = result_test.isnull().sum().sort_values(ascending=False)/len(result_test)
miss_value_percent = (miss_value_percent * 100).round(decimals=2)

if (miss_value_percent>60).any():
    delect_colomns = list(miss_value_percent[miss_value_percent>60].index)
    for n in delect_colomns:
        result_test = result_test.drop(columns=n)
        
# AMT_CREDIT空值只有一個直接補0
result_test['AMT_CREDIT'].replace({np.nan: 0}, inplace = True)

print(f'刪了欄位 {delect_colomns} ')

刪了欄位 ['RATE_INTEREST_PRIVILEGED', 'RATE_INTEREST_PRIMARY', 'DAYS_FIRST_DRAWING'] 


<h1> 特徵工程

In [17]:
# 2020-01-21

test = result_test.copy() # 先新建一個df

test['ANNUITY/CREDIT'] = test['AMT_ANNUITY'] / test['AMT_CREDIT']
test['DOWN_PAYMENT/ANNUITY'] = test['AMT_DOWN_PAYMENT'] / test['AMT_ANNUITY']
test['DOWN_PAYMENT/CREDIT'] = test['AMT_DOWN_PAYMENT'] / test['AMT_CREDIT']
test['DOWN_PAYMENT/ANNUITY'] = test['AMT_DOWN_PAYMENT'] / test['AMT_ANNUITY']
test['GOODS_PRICE/CREDIT'] = test['AMT_GOODS_PRICE'] / test['AMT_CREDIT']
test['APPLICATION/CREDIT'] = test['AMT_APPLICATION'] / test['AMT_CREDIT']
test['APPLICATION/GOODS_PRICE'] = test['AMT_APPLICATION'] / test['AMT_GOODS_PRICE']
test['DAYS_LAST_DUE-DAYS_TERMINATION'] = test['DAYS_LAST_DUE'] - test['DAYS_TERMINATION']

# 將有特殊狀態的欄位新增 (onehot)

onehot_list = ['NAME_CONTRACT_STATUS',
'NAME_TYPE_SUITE',
'CODE_REJECT_REASON',
'NAME_PAYMENT_TYPE',
'NAME_PRODUCT_TYPE',
'NFLAG_INSURED_ON_APPROVAL',
'PRODUCT_COMBINATION',
'NAME_SELLER_INDUSTRY',
'NAME_YIELD_GROUP']

test = pd.get_dummies(test, columns = onehot_list , dummy_na=True)

# *****************************************************************************************************************

# 針對previous_application 新增的欄位做計算

test1=test.copy()

# 把float64拿出來作為欄位要處理的鍵值對  ex: 'AMT_ANNUITY': ['sum', 'max', 'min', 'mean'] ,  ......'
new_feature = [
              'AMT_ANNUITY',
              'AMT_APPLICATION',
              'AMT_CREDIT',
              'AMT_DOWN_PAYMENT',
              'AMT_GOODS_PRICE',
              'DAYS_LAST_DUE',
              'DAYS_TERMINATION',
              'ANNUITY/CREDIT',
              'DOWN_PAYMENT/ANNUITY',
              'DOWN_PAYMENT/CREDIT',
              'DOWN_PAYMENT/ANNUITY',
              'GOODS_PRICE/CREDIT',
              'APPLICATION/CREDIT',
              'APPLICATION/GOODS_PRICE',
              'DAYS_LAST_DUE-DAYS_TERMINATION']

dict_type = {}
for i in new_feature:
        dict_type[i] = ['max','min']
num_aggregations = dict_type

# 計算出現的次數
# count = {'SK_ID_PREV':'count'}

test_1 = test.groupby(['SK_ID_CURR']).agg({**num_aggregations})


# 重新排列欄位
columns = []

for m in test_1.columns.levels[0]:
    for n in test_1.columns.levels[1]:
#         if m == 'SK_ID_PREV':
#             columns.append('count')
#             break
        columns.append(f'PREV_{m}_{n}')
        
test_1.columns = columns

test_1.reset_index(level=('SK_ID_CURR'),inplace=True)

# *****************************************************************************************************************

# 針對previous_application類別型態onehot後做計算

test2 = test.copy()

# 把onehot出來欄位變成要處理的鍵值對  ex:'NAME_CONTRACT_TYPE_Cash loans': ['count'] ,  ......'
dict_type = {}
for i in list(test.columns[91:]):
        dict_type[i] = ['sum']
num_aggregations = dict_type
# 計算出現的次數
count = {'SK_ID_PREV':'count'}

test_2 = test2.groupby(['SK_ID_CURR']).agg({**count,**num_aggregations})


# 重新排列欄位
columns = []

for m in test_2.columns.levels[0]:
    for n in test_2.columns.levels[1][1:]:
        if m == 'SK_ID_PREV':
            columns.append('count')
            break
        columns.append(f'PREV_{m}_rate')
        
test_2.columns = columns

test_2.reset_index(level=('SK_ID_CURR'),inplace=True)

# 計算比例
ID_count_t = test_2['count'].values.reshape((340893,1))
test_rate = (test_2.iloc[:,2:]/ID_count_t)

# 組在一起
test_2_rate = pd.concat([test_2['SK_ID_CURR'],test_rate], axis = 1)

# *****************************************************************************************************************

# SELLERPLACE_AREA集中程度(在同一家的最大出現次數除以此人的貸款數)

test4 = test.copy()
test4['Total_count'] = test4['SK_ID_PREV']
# 先以各ID及各分行去計次
test_4 = test4.groupby(['SK_ID_CURR','SELLERPLACE_AREA']).agg({'Total_count':'count'})
test_4.reset_index(level=('SK_ID_CURR','SELLERPLACE_AREA'),inplace=True)
# 新增一欄作為各ID在哪間分行使用作多的欄位
test_4['count_max'] =test_4['Total_count']
test_4 = test_4.groupby(['SK_ID_CURR']).agg({'count_max':'max','Total_count':'sum'})
# 新增一欄計算比例
test_4['SELLERPLACE_AREA_HIGH_rate'] = test_4['count_max']/test_4['Total_count']
test_4.reset_index(level=('SK_ID_CURR'),inplace=True)
# 只留比例
test_4 = test_4.drop(columns=['count_max','Total_count'],axis=1
                     


In [49]:
file_list = [test_1,test_2_rate,test_4]

# 建一個空的df
second_result = pd.DataFrame(columns=['SK_ID_CURR'])

for i in file_list:    
    second_result = pd.merge(second_result, i,how='outer')

<h1>合併

In [67]:
# 最後合併

Hl0 = pd.read_csv(f'..\\..\\Desktop\\home-credit-default-risk\\C-application_train_cleaning_with_bureau.csv')
Hl1 = pd.read_csv(f'..\\..\\Desktop\\home-credit-default-risk\\C-previous_application_Aiyd.csv')
Hl2 = pd.read_csv(f'..\\..\\Desktop\\home-credit-default-risk\\C-previous_application_FE_w_installment&POS.csv')
Hl3 = pd.read_csv(f'..\\..\\Desktop\\home-credit-default-risk\\C-previous_application_FE_w_installment&POS_CL.csv')
Hl4 = pd.read_csv(f'..\\..\\Desktop\\home-credit-default-risk\\C-previous_application_FE_w_installment&POS_in2y.csv')
Hl5 = pd.read_csv(f'..\\..\\Desktop\\home-credit-default-risk\\C-previous_application_FE_w_installment&POS_RL.csv')
Hl6 = pd.read_csv(f'..\\..\\Desktop\\home-credit-default-risk\\C-previous_application_FE_w_CreditCard.csv')

Hl0 = Hl0.drop(columns=['Unnamed: 0'])
Hl1 = Hl1.drop(columns=['Unnamed: 0'])

file_list = [Hl0,Hl1,Hl2,Hl3,Hl4,Hl5,Hl6]

# 建一個空的df
final_result = pd.DataFrame(columns=['SK_ID_CURR'])

for i in file_list:
    if len(i.columns) ==335:
        final_result = i
    final_result = pd.merge(final_result, i,how='left')



In [68]:
final_result.shape

(356255, 796)

In [80]:
Hl0.shape

(356255, 335)

In [72]:
final_result.to_csv('..\\..\\Desktop\\home-credit-default-risk\\BDSE12_03G_HomeCredit_V1.csv')